## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-12-15-51-55 +0000,nypost,Texas Target shooting suspect Ethan Nieneker s...,https://nypost.com/2025/08/12/us-news/texas-ta...
1,2025-08-12-15-48-32 +0000,nyt,Trump Promised a Golden Age. Then a Montana Lu...,https://www.nytimes.com/2025/08/12/us/trump-pl...
2,2025-08-12-15-45-03 +0000,wapo,MedStar copter unit changed protocols after Po...,https://www.washingtonpost.com/business/2025/0...
3,2025-08-12-15-45-00 +0000,wsj,The White House plans to conduct a far-reachin...,https://www.wsj.com/politics/policy/white-hous...
4,2025-08-12-15-42-22 +0000,wapo,Trump nominates Bureau of Labor Statistics cri...,https://www.washingtonpost.com/business/2025/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,67
275,china,16
62,up,15
58,new,13
87,will,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
170,2025-08-12-01-47-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...,141
243,2025-08-11-19-43-00 +0000,wsj,The Trump administration’s decision to charge ...,https://www.wsj.com/politics/national-security...,138
288,2025-08-11-16-09-47 +0000,nypost,Trump says he’ll tell Putin to end Ukraine inv...,https://nypost.com/2025/08/11/world-news/trump...,132
246,2025-08-11-19-33-26 +0000,nypost,Trump extends China tariff deadline for 90 day...,https://nypost.com/2025/08/11/world-news/trump...,119
219,2025-08-11-21-25-43 +0000,bbc,Trump says he will try to get back territory f...,https://www.bbc.com/news/articles/c0e99yqv332o...,119


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
170,141,2025-08-12-01-47-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...
243,64,2025-08-11-19-43-00 +0000,wsj,The Trump administration’s decision to charge ...,https://www.wsj.com/politics/national-security...
288,55,2025-08-11-16-09-47 +0000,nypost,Trump says he’ll tell Putin to end Ukraine inv...,https://nypost.com/2025/08/11/world-news/trump...
76,50,2025-08-12-11-49-16 +0000,bbc,Who were the Al Jazeera journalists killed by ...,https://www.bbc.com/news/articles/c6200wnez73o...
278,41,2025-08-11-17-06-32 +0000,bbc,Harry and Meghan sign new multi-year Netflix deal,https://www.bbc.com/news/articles/cvgnne6lznzo...
184,40,2025-08-12-00-10-09 +0000,nypost,ABC News anchor Kyra Phillips reveals she was ...,https://nypost.com/2025/08/11/us-news/abc-news...
3,37,2025-08-12-15-45-00 +0000,wsj,The White House plans to conduct a far-reachin...,https://www.wsj.com/politics/policy/white-hous...
57,36,2025-08-12-13-11-43 +0000,nypost,Mom allegedly open fired to break up fight at ...,https://nypost.com/2025/08/12/us-news/nc-mom-a...
73,34,2025-08-12-12-00-00 +0000,nypost,Ex-NYPD cop claims Bill de Blasio ‘didn’t like...,https://nypost.com/2025/08/12/us-news/ex-nypd-...
258,30,2025-08-11-18-44-07 +0000,bbc,Jellyfish swarm forces French nuclear plant to...,https://www.bbc.com/news/articles/cx299eyg7qko...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
